In [1]:
# This is required to run multiple processes on Unity for some reason.
from multiprocessing import set_start_method
set_start_method('spawn', force=True)


import os
# os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'
os.environ['XLA_FLAGS'] = '--xla_gpu_enable_command_buffer='
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.7'
# os.environ['JAX_PLATFORMS'] = 'cpu'

In [2]:
%matplotlib widget
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import jax
from pathlib import Path
from importlib import reload

src = str(Path('../src').resolve())
if src not in sys.path:
    sys.path.append(src)
import config, data, models, train, evaluate

In [3]:
# reload(config)
from config import read_config

cfg, cfg_str = read_config("../runs/swotq/64.yml")
cfg['num_workers'] = 1
cfg['log'] = True
cfg['quiet'] = False


In [6]:
reload(data)
from data import HydroDataset

dataset = HydroDataset(cfg)

/work/pi_kandread_umass_edu/tss-ml/src/data/hydrodata.py:174: UserWarning: Dropping numerical attributes with 0 variance or NaN values: ['wet_pc_s06', 'wet_pc_s05', 'glc_pc_s17', 'glc_pc_s05', 'glc_pc_s07', 'glc_pc_s08', 'glc_pc_s19', 'glc_pc_s03', 'n_bas_up', 'pnv_pc_s01', 'wet_pc_s07', 'aspect', 'area', 'order', 'pnv_pc_s03']
  warnings.warn(f"Dropping numerical attributes with 0 variance or NaN values: {cols_to_drop}", UserWarning)


Data Hash: 5df32cf3b1d0d221c4e77f204df907bdf0b2cd900473cc0db544e3084a287d6a
Using cached basin dataset file.


Updating Indices:   0%|          | 0/565 [00:00<?, ?it/s]

In [ ]:
# reload(config)
from config import set_model_data_args
from data import HydroDataLoader

reload(train)
from train import Trainer

cfg = set_model_data_args(cfg, dataset)

dataloader = HydroDataLoader(cfg, dataset)
trainer = Trainer(cfg, dataloader)
trainer.start_training()

In [ ]:
plt.close('all')
plt.imshow(batch['swot'][3])

In [ ]:
list(dataset.scale.keys())

In [ ]:
batch['swot'][3,8,:]

In [ ]:
alt_cfg = cfg.copy()
alt_cfg['batch_size'] = 1
dataloader = TAPDataLoader(alt_cfg, dataset)

for basin, date, batch in dataloader:
    break
    
single_data = {k: v[0,...] for k, v in batch.items()}

In [ ]:
trainer.model(single_data, jax.random.PRNGKey(0))

In [ ]:
batch['y'][0,...].shape

In [ ]:
# Resume training. Either directly from memory or loading a checkpoint.
import optax

# trainer.load_state('epoch100')
# trainer.load_last_state()

more_epochs = 0
new_schedule = optax.exponential_decay(0.01, trainer.epoch+more_epochs, 0.001, transition_begin=trainer.epoch)
trainer.lr_schedule = new_schedule
trainer.num_epochs += more_epochs

#Have to make a new dataloader when the last one is interrupted. 
trainer.dataloader = TAPDataLoader(cfg, dataset) 
trainer.start_training() 

In [ ]:
reload(evaluate)
from evaluate import predict, get_all_metrics

# basin = np.random.choice(dataset.basins).tolist()
basin = 'USGS-09367540'

cfg['data_subset'] = 'predict'
cfg['basin_subset'] =  basin
cfg['num_workers'] = 0 # Faster for small runs
dataloader = TAPDataLoader(cfg, dataset)

results = predict(trainer.model, dataloader, seed=0, denormalize=True)
results['pred'] = results['pred'] * (results['pred']>0) #Clip predictions to 0

results = results.reset_index()
results = results.sort_values(by='date')
results = results.drop(columns=['basin'], axis=1, level=0)
results.set_index('date', inplace=True)


In [ ]:
feature = 'usgs_q'

# Plot the true values and predictions
fig, ax = plt.subplots(figsize=(12, 6))
results['pred'][feature].plot(ax=ax)
results['obs'][feature].plot(ax=ax,linestyle='None',marker='.')

plt.title(f"Basin: {basin}")
plt.legend()
fig.autofmt_xdate()
# plt.ylim([0,20000])
plt.show()

In [ ]:
basin

In [ ]:
"""
'USGS-09367540'
"""

In [ ]:
results.plot.scatter('obs','pred')
plt.gca().axis('square')
# plt.xscale('log')
# plt.yscale('log')
# plt.xlim([0,20])
# plt.ylim([0,20])
plt.show()

In [ ]:
import train
from data import TAPDataset, TAPDataLoader

state_dir = Path("../runs/notebook/20240603_1359/epoch18")
cfg, model, trainer_state, opt_state = train.load_state(state_dir)
dataset = TAPDataset(cfg)

In [ ]:
reload(evaluate)
from evaluate import predict, get_all_metrics

cfg['data_subset'] = 'test'
cfg['num_workers'] = 4
dataloader = TAPDataLoader(cfg, dataset)

results = predict(model, dataloader, seed=0, denormalize=True)
results['pred'] = results['pred']# * (results['pred']>0) #Clip predictions to 0

# results = results.reset_index()
# results = results.sort_values(by='date')

metrics = get_all_metrics(results['obs'],results['pred'])
metrics

In [ ]:
%matplotlib widget
plt.close('all')
plt.scatter(batch['y'][...,-1],pred[...,-1])
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10,3))
xd = axes[0].imshow(batch['x_dd'][:,:,0],aspect='auto')
fig.colorbar(xd, ax=axes[0])
xs = axes[1].imshow(batch['x_s'],aspect='auto')
fig.colorbar(xs, ax=axes[1]) 

In [ ]:
batch['x_dd'][:,:,0].shape

In [ ]:
basins[idx_max_err]

In [ ]:
positional_encoding = trainer.model.d_encoder.embedder.positional_encoding

plt.figure(figsize=(10, 8))
plt.imshow(positional_encoding, cmap='viridis')
plt.xlabel('Embedding Dimension')
plt.ylabel('Position')
plt.title('Positional Encodings')
plt.show()